# Módulo 7: Usar herramientas MCP locales en un agente de IA

En este ejercicio, crearás un servidor MCP local que expone herramientas para consultar el inventario y las ventas de componentes aeroespaciales. Luego, un agente cliente se conectará a este servidor para responder preguntas.

### Crear el servidor MCP local

Primero, creamos un archivo `server.py` que define las herramientas `get_inventory_levels` y `get_weekly_sales` usando `FastMCP`. Este archivo se ejecutará como un proceso separado.

In [ ]:
server_code = """
from mcp.server.fastmcp import FastMCP

mcp = FastMCP("AeroInventory")

@mcp.tool()
def get_inventory_levels() -> dict:
    \"\"\"Devuelve el inventario actual de componentes aero clave en las plantas.\"\"\"
    return {
        "Álabe de turbina (AP)": 6,
        "Carcasa del compresor": 8,
        "Conjunto de sello": 28,
        "Álabe guía de tobera": 5,
        "Carcasa del rodamiento": 12,
        "Eje": 9,
        "Difusor": 30,
        "Revestimiento del combustor": 3,
        "Bastidor frontal": 17,
        "Carcasa del ventilador": 45,
    }

@mcp.tool()
def get_weekly_sales() -> dict:
    \"\"\"Devuelve el consumo semanal por órdenes de trabajo de la última semana.\"\"\"
    return {
        "Álabe de turbina (AP)": 22,
        "Carcasa del compresor": 18,
        "Conjunto de sello": 3,
        "Álabe guía de tobera": 2,
        "Carcasa del rodamiento": 14,
        "Eje": 19,
        "Difusor": 4,
        "Revestimiento del combustor": 1,
        "Bastidor frontal": 13,
        "Carcasa del ventilador": 17,
    }

if __name__ == "__main__":
    mcp.run()
"""

print("Creating local MCP server file...")
with open("local_MCP_server.py", "w") as f:
    f.write(server_code)
print("Server file 'local_MCP_server.py' created successfully")


### Cargar librerías y variables de entorno para el cliente

In [ ]:
import asyncio
import os
from dotenv import load_dotenv, find_dotenv
from agent_framework import ChatAgent, MCPStdioTool
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential

load_dotenv(find_dotenv(usecwd=True))

### Crear y ejecutar el agente cliente

El `ChatAgent` se conecta al servidor MCP local usando `MCPStdioTool`, que inicia el proceso `python server.py`. Luego, podemos hacerle preguntas al agente sobre el inventario.

In [ ]:
async def query_inventory(prompt):
    print("Starting local MCP server...")
    async with (
        AzureCliCredential() as credential,
        MCPStdioTool(
            name="aeroinventory",
            command="python",
            args=["local_MCP_server.py"],  # Use file created in previous cell
        ) as mcp_server,
        ChatAgent(
            chat_client=AzureAIAgentClient(async_credential=credential),
            name="InventoryAgent",
            instructions=(
                "Eres un asistente de gestión de inventario aeronáutico que puede consultar niveles de stock y ventas semanales."
            ),
        ) as agent,
    ):
        print("Local MCP server connected")
        print(f"Running inventory query: {prompt}")
        result = await agent.run(prompt, tools=mcp_server)
        print("Query result:")
        print(result)


In [ ]:

# Enter your question here
print("Starting inventory analysis...")
user_prompt = "Lista los componentes con inventario < 15 y su consumo semanal."
await query_inventory(user_prompt)
print("Inventory analysis completed")

In [ ]:

# Enter your question here
print("Starting inventory analysis...")
user_prompt = "¿Qué piezas podrían agotarse esta semana considerando el consumo semanal y el inventario actual?"
await query_inventory(user_prompt)
print("Inventory analysis completed")